# Data Cleaning and Feature Engineering -ET_project




### Libraries

In [1]:
import pandas as pd
import numpy as np
import warnings

#### Read DATASETS(CSVS)

In [2]:
df= pd.read_csv('/Users/miguelaraujo/code/MrAraujo99/ET_Predictor/raw_data/UFO sightings/scrubbed.csv')

/var/folders/0m/p33mzd1n3nq5v3977g7mxf1r0000gn/T/ipykernel_3783/3933699196.py:1: DtypeWarning: Columns (5,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df= pd.read_csv('/Users/miguelaraujo/code/MrAraujo99/ET_Predictor/raw_data/UFO sightings/scrubbed.csv')


In [3]:
df.shape

(80332, 11)

In [4]:
df.columns

Index(['datetime', 'city', 'state', 'country', 'shape', 'duration (seconds)',
       'duration (hours/min)', 'comments', 'date posted', 'latitude',
       'longitude '],
      dtype='object')

#### Display Initial Data for Each DataFrame and drop columns you dont need

In [5]:
df.head(1)

,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude
0,10/10/1949 20:30,san marcos,tx,us,cylinder,2700,45 minutes,This event took place in early fall around 194...,4/27/2004,29.8830556,-97.941111


In [6]:
df.dtypes

datetime                 object
city                     object
state                    object
country                  object
shape                    object
duration (seconds)       object
duration (hours/min)     object
comments                 object
date posted              object
latitude                 object
longitude               float64
dtype: object

In [7]:
df.columns

Index(['datetime', 'city', 'state', 'country', 'shape', 'duration (seconds)',
       'duration (hours/min)', 'comments', 'date posted', 'latitude',
       'longitude '],
      dtype='object')

In [8]:
df = df.drop(columns = ['comments', 'duration (hours/min)', 'duration (seconds)', 'date posted'])

In [9]:
df.head()

,datetime,city,state,country,shape,latitude,longitude
0,10/10/1949 20:30,san marcos,tx,us,cylinder,29.8830556,-97.941111
1,10/10/1949 21:00,lackland afb,tx,NaN,light,29.38421,-98.581082
2,10/10/1955 17:00,chester (uk/england),NaN,gb,circle,53.2,-2.916667
3,10/10/1956 21:00,edna,tx,us,circle,28.9783333,-96.645833
4,10/10/1960 20:00,kaneohe,hi,us,light,21.4180556,-157.803611


In [10]:
# Convert 'datetime' to datetime objects
df['datetime']=pd.to_datetime(df['datetime'], errors='coerce')

In [11]:
# Convert 'latitude' and 'longitude' to numeric
df['latitude'] = pd.to_numeric(df['latitude'], errors='coerce')
df['longitude '] = pd.to_numeric(df['longitude '], errors='coerce')

In [12]:
# Check for NaN values in the dataframe
nan_summary = df.isna().sum()

# Display the summary of NaN values in each column
print("Summary of NaN values in each column:")
print(nan_summary)

Summary of NaN values in each column:
datetime       694
city             0
state         5797
country       9670
shape         1932
latitude         1
longitude        0
dtype: int64


In [ ]:
# Function to determine season
def get_season(month):
    if month in [12, 1, 2]:
        return 'winter'  # Winter
    elif month in [3, 4, 5]:
        return 'Spring'  # Spring
    elif month in [6, 7, 8]:
        return 'Summer'  # Summer
    elif month in [9, 10, 11]:
        return 'Fall'  # Fall